# Monte Carlo

This notebook creates three files. For a given number of market situations and a given set of prices, it creates features and saves them in the `demand_prediction_data` file. Next, for each market situation and each price, the notebook runs multiple simulations and tracks purchases. The notebook then calculates the probabilities of a purchase for each price and market situation and stores them in the `PEW_comparison` file.

After this, for one price, the time between adjustments of prices gets split into B parts and for each purchase the time slot o fit is tracked. At the end, it outputs a probability for each time slot, which corresponds to how likely a purchase is in this time slot after a price adjustment. The probabilities are saved in the `cond_prob_out` file. The time horizon is 1.

Because of performance reasons the number of simulations and the number of considered market situations used in the start configuration are smaller than the ones used for the data for the paper. The inline comments show the values used in the paper, which lead to a longer runtime.

In [ ]:
import numpy as np
import random
import csv

In [ ]:
numberPlayers = 5
timeHorizon = 1
numberMarketReactionEvents = round(1.1*timeHorizon)
meanTimeBetweenCustomerArrival = 2
numberOfCustomerEvents = round(1.1*timeHorizon/meanTimeBetweenCustomerArrival)
prices = np.arange(0,15.1,0.1)
watchedCompany = 0
numberSimulations = 100  # value used for paper: 10000
numberConsideredMarketSituations = 10   # value used for paper: 40
m = 2
numberCustomerEvents = 4
reactionTimeCompany = timeHorizon
intensity_price = 7
price_index = np.where(prices== intensity_price)[0][0]
bb = 20

In [ ]:
pricesCompanies = np.array([round(random.uniform(6,16),2) for _ in range(numberPlayers)])
qualityCompanies = np.array([round(random.uniform(0.5,5.5)) for _ in range(numberPlayers)])
ratingCompanies = np.array([round(random.uniform(90,100),1) for _ in range(numberPlayers)])

In [ ]:
def explanatoryVariables(explanatoryVariable, numberPrice, marketSituation):
    return {
        '0' : 1,
        '1' : priceRanksForFirm[marketSituation][numberPrice],
        '2' : 1 if priceRanksForFirm[marketSituation][numberPrice]==1 else 0,
        '3' : qualityRankForFirm[marketSituation],
        '4' : ratingRanks[marketSituation][watchedCompany],
        '5' : prices[numberPrice],
        '6' : prices[numberPrice] - min([pricesCompaniesPred[j][marketSituation] for j in range(numberPlayers) if j!=watchedCompany]) if numberPlayers > 1 else 0,
        '7' : qualityCompaniesPred[watchedCompany][marketSituation],
        '8' : ratingCompaniesPred[watchedCompany][marketSituation],
        '9' : 15 - (prices[numberPrice]+0.5*qualityCompaniesPred[watchedCompany][marketSituation]+0.25*(100-ratingCompaniesPred[watchedCompany][marketSituation])) if(
        (prices[numberPrice]+0.5*qualityCompaniesPred[watchedCompany][marketSituation]+0.25*(100-ratingCompaniesPred[watchedCompany][marketSituation])) <
        min([pricesCompaniesPred[j][marketSituation] +0.5*qualityCompaniesPred[j][marketSituation]+0.25*(100-ratingCompaniesPred[j][marketSituation]) for j in range(numberPlayers) if j!= watchedCompany]))
        else 0,
    }[str(explanatoryVariable)]

In [ ]:
pricesCompaniesPred = np.array([[round(random.uniform(6,16),2) for _ in range(numberConsideredMarketSituations)]  for _ in range(numberPlayers)])
qualityCompaniesPred = np.array([[round(random.uniform(0.5,5.5)) for _ in range(numberConsideredMarketSituations)]  for _ in range(numberPlayers)])
ratingCompaniesPred = np.array([[round(random.uniform(90,100),1) for _ in range(numberConsideredMarketSituations)] for _ in range(numberPlayers)])

priceRanksForFirm = np.array([[1 +  
              sum([1 for i in range(numberPlayers) if prices[j]> pricesCompaniesPred[i][w] and i!=watchedCompany]) +
              sum([1 for i in range(numberPlayers) if prices[j]== pricesCompaniesPred[i][w] and i!=watchedCompany])/2
              for j in range(len(prices))]for w in range(numberConsideredMarketSituations)])
qualityRankForFirm = np.array([(1 +  
              sum([1 for i in range(numberPlayers) if qualityCompaniesPred[watchedCompany][w]> qualityCompaniesPred[i][w] and i!=watchedCompany]) +
              sum([1 for i in range(numberPlayers) if qualityCompaniesPred[watchedCompany][w]== qualityCompaniesPred[i][w] and i!=watchedCompany])/2)for w in range(numberConsideredMarketSituations)])
                         
ratingRanks = np.array([[1 +  
              sum([1 for i in range(numberPlayers) if ratingCompaniesPred[j][w]> ratingCompaniesPred[i][w] and i!=j]) +
              sum([1 for i in range(numberPlayers) if ratingCompaniesPred[j][w]== ratingCompaniesPred[i][w] and i!=j])/2
              for j in range(numberPlayers)]for w in range(numberConsideredMarketSituations)])

numberExplanatoryVariables = 10

valuesExplanatoryVariables = [[[explanatoryVariables(explanatoryVariable, numberPrice, w) for explanatoryVariable in range(numberExplanatoryVariables)] for numberPrice in range(len(prices))]for w in range(numberConsideredMarketSituations)]
predictionEstimated = [[0.5 for _ in range(len(prices))] for _ in range(numberConsideredMarketSituations)]

with open('demand_prediction_data_S3.csv', 'w') as file:
    writer = csv.writer(file, delimiter='\t')
    for marketSituation in range(numberConsideredMarketSituations):
        for numberPrice in range(len(prices)):
            line = list(valuesExplanatoryVariables[marketSituation][numberPrice])
            line.remove(1)
            line.insert(0, prices[numberPrice])
            line.insert(0, marketSituation+1)
            writer.writerow(line)

In [ ]:
def calculateTimeOfAdjustmentsCompany(company, simulationNumber, i, timeBetweenMarketEventsCompany):
    if i==0:
        if company != watchedCompany:
            return(random.uniform(0,1))
        else:
            return(reactionTimeCompany)
    else:
        return(calculateTimeOfAdjustmentsCompany(company, simulationNumber, i-1, timeBetweenMarketEventsCompany)+ timeBetweenMarketEventsCompany[simulationNumber][company][i])
    
def calculateTimeOfCustomerArrival(customer, simulationNumber, timeBetweenMarketEventsCustomer):
    if customer==0:
        return 0
    else:
        return(calculateTimeOfCustomerArrival(customer-1, simulationNumber, timeBetweenMarketEventsCustomer) + timeBetweenMarketEventsCustomer[simulationNumber][customer])
    
def priceAdjustmentRules(c, pricesCompanies):
    return {
        '0' : pricesCompanies[0],
        '1' : 9 if pricesCompanies[c]<4 else round(min([max(3.9, pricesCompanies[company]-0.5) for company in range(numberPlayers) if company != c]),2),
        '2' : 12 if pricesCompanies[c]<6 else round(min([max(5.9, pricesCompanies[company]-0.5) for company in range(numberPlayers) if company != c]),2),
        '3' : round(random.uniform(11,11),2),
        '4' : round(random.uniform(13,13),2)
    }[str(c)]

def calculatePrices(pricesCompaniesAdjustments, adjustingFirm):
    for i in range(len(pricesCompaniesAdjustments)):
        if i > 0:
            pricesCompaniesAdjustments[i][adjustingFirm[i]] = priceAdjustmentRules(adjustingFirm[i],pricesCompaniesAdjustments[i-1])
    return pricesCompaniesAdjustments

In [ ]:
timeBetweenMarketEventsCompany = np.array([[[random.uniform(0.8,1.2) if company!= watchedCompany else reactionTimeCompany for i in range(m)] for company in range(numberPlayers)] for _ in range(numberSimulations)])
timeOfAdjustmentsCompany = np.array([[[calculateTimeOfAdjustmentsCompany(company,simulationNumber ,i,timeBetweenMarketEventsCompany) for i in range(m)] for company in range(numberPlayers)] for simulationNumber in range(numberSimulations)])

priceAdjustmentsBeforeHorizon = np.array([set([timeOfAdjustmentsCompany[simulationNumber].flatten()[i] for i in range(len(timeOfAdjustmentsCompany[simulationNumber].flatten())) if timeOfAdjustmentsCompany[simulationNumber].flatten()[i] < timeHorizon])for simulationNumber in range(numberSimulations)])
priceAdjustmentsBeforeHorizon = np.array([sorted(priceAdjustmentsBeforeHorizon[simulationNumber]) for simulationNumber in range(numberSimulations)])

numberPriceAdjustmentsBeforeHorizon = np.array([len(priceAdjustmentsBeforeHorizon[simulationNumber]) for simulationNumber in range(numberSimulations)])

timeMarketEvents = np.array([[0 if i==0 else priceAdjustmentsBeforeHorizon[simulationNumber][i-1] for i in range(numberPriceAdjustmentsBeforeHorizon[simulationNumber]+1)]for simulationNumber in range(numberSimulations)])
timeBetweenMarketEventsCustomer = np.array([[timeMarketEvents[simulationNumber][i+1]-timeMarketEvents[simulationNumber][i] for i in range(numberPriceAdjustmentsBeforeHorizon[simulationNumber])] for simulationNumber in range(numberSimulations)])

timeBetweenCustomerEvents = np.array([[meanTimeBetweenCustomerArrival*np.random.exponential() for _ in range(numberCustomerEvents)]for simulationNumber in range(numberSimulations)])
timeCustomerEvents = np.array([[calculateTimeOfCustomerArrival(customer, simulationNumber, timeBetweenMarketEventsCustomer) for customer in range(numberCustomerEvents)] for simulationNumber in range(numberSimulations)])
adjustingFirm = np.array([[min(np.where(timeOfAdjustmentsCompany[simulationNumber]==priceAdjustmentsBeforeHorizon[simulationNumber][i] )[0]) for i in range(numberPriceAdjustmentsBeforeHorizon[simulationNumber])] for simulationNumber in range(numberSimulations)])

randomCustomerScoringQuality = np.array([[random.uniform(0,1) for _ in range(numberCustomerEvents)] for simulationNumber in range(numberSimulations)])
randomCustomerScoringRating = np.array([[random.uniform(0,0.5) for _ in range(numberCustomerEvents)] for simulationNumber in range(numberSimulations)])
                         
customerScoringQualityRating = np.array([[[[(randomCustomerScoringQuality[s][c]*qualityCompaniesPred[k,w] +
    randomCustomerScoringRating[s][c]*(100-ratingCompaniesPred[k,w]))for k in range(numberPlayers)] for c in range(numberCustomerEvents)] for s in range(numberSimulations)]for w in range(numberConsideredMarketSituations)])
pricesCompanies = np.array([[[pricesCompaniesPred[k,w] if k!=watchedCompany else prices[a] for w in range(numberConsideredMarketSituations)] for a in range(len(prices))]  for k in range(numberPlayers)])
pricesCompaniesAdjustments = np.array([[[[[pricesCompanies[k,a,w]  for k in range(numberPlayers)]for _ in range(numberPriceAdjustmentsBeforeHorizon[s])] for s in range(numberSimulations)] for w in range(numberConsideredMarketSituations)] for a in range(len(prices))])
pricesCompaniesAdjustments = np.array([[[np.array(calculatePrices(pricesCompaniesAdjustments[a,w,s], adjustingFirm[s])) for s in range(numberSimulations)] for w in range(numberConsideredMarketSituations)] for a in range(len(prices))])

gen = np.array([[[[[x for x in range(numberCustomerEvents) if timeMarketEvents[s][i]< timeCustomerEvents[s][x]< timeMarketEvents[s][i+1]] for i in range(numberPriceAdjustmentsBeforeHorizon[s])] for s in range(numberSimulations)] for w in range(numberConsideredMarketSituations)] for a in range(len(prices))])
randomCustomerScoring = np.array([[[[[[(pricesCompaniesAdjustments[a,w,s][i,k] + customerScoringQualityRating[w,s,c,k]) for k in range(numberPlayers)]for c in np.squeeze(gen[a,w,s])[i]]for i in range(numberPriceAdjustmentsBeforeHorizon[s])] for s in range(numberSimulations)] for w in range(numberConsideredMarketSituations)] for a in range(len(prices))])
randomCustomerDecision = np.array([[[[[np.argmin(randomCustomerScoring[a,w,s][i][c]) if min(randomCustomerScoring[a,w,s][i][c])< random.uniform(5,15) else -1 for c in range(len(np.squeeze(gen[a,w,s])[i]))]for i in range(numberPriceAdjustmentsBeforeHorizon[s])] for s in range(numberSimulations)] for w in range(numberConsideredMarketSituations)] for a in range(len(prices))])
As = np.array([[[[np.count_nonzero(np.hstack(randomCustomerDecision[a,w,s])== k) for k in range(numberPlayers)]for s in range(numberSimulations)] for w in range(numberConsideredMarketSituations)] for a in range(len(prices))])

A = np.array([[[[1/numberSimulations*np.count_nonzero(As[a,w,:,k]==x) for k in range(numberPlayers)]for x in range(6)]for w in range(numberConsideredMarketSituations)] for a in range(len(prices))])
P_ori = [[1- A[a,w,0,watchedCompany]for w in range(numberConsideredMarketSituations)] for a in range(len(prices))]
P_ori2 = [[sum([A[a,w,2,watchedCompany]for x in range(2,6)])for w in range(numberConsideredMarketSituations)] for a in range(len(prices))]

EW_ori = [[prices[a]*P_ori[a][w] for w in range(numberConsideredMarketSituations)]for a in range(len(prices))]
EW_est = [[prices[a]*predictionEstimated[w][a] for w in range(numberConsideredMarketSituations)]for a in range(len(prices))]

D_P = [1/numberConsideredMarketSituations*sum([abs(P_ori[a][w]- predictionEstimated[w][a]) for w in range(numberConsideredMarketSituations)]) for a in range(len(prices))]
D_EW = [1/numberConsideredMarketSituations*sum([abs(EW_ori[a][w]- EW_est[a][w]) for w in range(numberConsideredMarketSituations)]) for a in range(len(prices))]

with open('PEW_comparison_S3.csv', 'w') as file:
    writer = csv.writer(file, delimiter='\t')
    for w in range(numberConsideredMarketSituations):
        for a in range(len(prices)):
            line = [w+1,prices[a],P_ori[a][w], P_ori2[a][w], predictionEstimated[w][a], EW_ori[a][w], EW_est[a][w], D_P[a], D_EW[a]]
            writer.writerow(line)

w = 0

b = np.array([[[int(round((timeCustomerEvents[s][c]+1/bb/2)*bb))for c in range(len(gen[price_index,w,s][i]))]for i in range(numberPriceAdjustmentsBeforeHorizon[s])] for s in range(numberSimulations)])

At = np.array([[sum([sum([sum([1 if b[s][i][c]==t and randomCustomerDecision[price_index,w,s][i][c]==k  else 0 for c in range(len(gen[price_index,w,s][i]))])for i in range(numberPriceAdjustmentsBeforeHorizon[s])])for s in range(numberSimulations)])for k in range(numberPlayers)]for t in range(reactionTimeCompany*bb+1)])

with open('cond_prob_out_S3.csv', 'w') as file:
    writer = csv.writer(file, delimiter='\t')
    for t in range(reactionTimeCompany*bb+1):
        line = [t/bb, At[t][watchedCompany]/numberSimulations]
        writer.writerow(line)